# Automated Prompting

Automated prompting is an advanced technique in prompt engineering, introduced by [Zhang et al. (2022)](https://arxiv.org/abs/2211.01910), that utilizes a GenAI model to automatically generate and evaluate well structured prompts to be used within the same or other models.

For the purposes of this experiment, we are only creating a single automated prompt in a zero shot style using meta prompting.

In [ ]:
##
## AUTOMATED PROMPTING
##

from _pipeline import create_payload, model_req

#### (1) Create an inbound prompt
MESSAGE = "A study companion discord bot that is to accept any document, analyse it, and answer questions based on the document."

#### (2) Simulate a Workflow Template
TEMPLATE_BEFORE="You are an expert in software requirement analysis. Your task is to generate a well-structured zero-shot prompt that guides an AI model to analyze and extract key requirements for a following solution:"
TEMPLATE_AFTER= ("The generated prompt must instruct the AI to Identify User Interactions (List all distinct interactions that users will perform within the system) " 
                 "and Define Supporting Functionalities (Specify the essential system functionalities required to enable these interactions). "
                 "The generated prompt must exclude non-essential details such as system architecture, implementation details, or non-functional requirements, "
                 "and be formatted as a standalone prompt with no additional explanation, introductory text, or metadata. Output only the generated prompt and nothing else.")
PROMPT = TEMPLATE_BEFORE + '\n' + MESSAGE + '\n' + TEMPLATE_AFTER

#### (3) Create a first payload
payload = create_payload(target="ollama",
                         model="llama3.2:latest", 
                         prompt=PROMPT, 
                         temperature=0.3, # Responsible for how "random" can the answer be. Keeping at 0.3 to keep prompt consistant yet allow for small variations.
                         num_ctx=512, # Essentially responsible for models memory / context window. 512 allows the model to create a consize response with some extra room if needed.
                         num_predict=250) # Responsible for how many word tokens are permitted to be returned. 250 keeps the prompt consize.

# Send out to the first model
time, response = model_req(payload=payload)
print("Automatically Generated Prompt:\n" + response)
if time: print(f'Time taken: {time}s')

print("\n\n\n")

#### (4) Create a second payload
automatedPayload = create_payload(target="ollama",
                                model="llama3.2:latest", 
                                prompt=response, 
                                temperature=0.2, # Responsible for how "random" can the answer be. Keeping at 0.2 to keep answers consistant
                                num_ctx=1024, # Essentially responsible for models memory / context window. 1024 ensures the model fully understands both the meta-prompt and the requirement analysis.
                                num_predict=500) # Responsible for how many word tokens are permitted to be returned. 500 keeps the answer consize and to the point.

# Send out to the second model
automatedTime, automatedResponse = model_req(payload=automatedPayload)
print("Model Response:\n" + automatedResponse)
if automatedTime: print(f'Time taken: {automatedTime}s')



{'model': 'llama3.2:latest', 'prompt': 'You are an expert in software requirement analysis. Your task is to generate a well-structured zero-shot prompt that guides an AI model to analyze and extract key requirements for a following solution:\nA study companion discord bot that is to accept any document, analyse it, and answer questions based on the document.\nThe generated prompt must instruct the AI to Identify User Interactions (List all distinct interactions that users will perform within the system) and Define Supporting Functionalities (Specify the essential system functionalities required to enable these interactions). The generated prompt must exclude non-essential details such as system architecture, implementation details, or non-functional requirements, and be formatted as a standalone prompt with no additional explanation, introductory text, or metadata. Output only the generated prompt and nothing else.', 'stream': False, 'options': {'temperature': 0.3, 'num_ctx': 512, 'num